In [1]:
import ee
import os
import time
import glob
import logging
import requests
import threading
import numpy as np
import pandas as pd
from tqdm import tqdm
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
# Authenticate to earth engine

ee.Authenticate()

Enter verification code:  4/1AX4XfWg9fnaqjEIDRSBxNZXH51Cw4Wmpeow7dH_ZvFg9OiNEreIehf90pn0



Successfully saved authorization token.


In [7]:
# Initialize earth engine connection

ee.Initialize()

In [170]:
# Set destination folder paths for results and figures
results_dir = os.path.join(os.getcwd().replace('/code', ''), 'results/unions')
fig_dir = os.path.join(os.getcwd().replace('/code', ''), 'figures')

In [144]:
# Import image and feature colelctions
s1 = ee.ImageCollection("COPERNICUS/S1_GRD")
unions = ee.FeatureCollection('projects/ee-devapatel3-bgdfloods/assets/gadm36_BGD_4')

#basins = ee.FeatureCollection("users/coreyscher/se-asia_subbasin_01min")
major_basins = ee.FeatureCollection("users/coreyscher/se-asia_basins_01min")
lulc = ee.Image('users/coreyscher/all_lcmaps_v3_2018_proj')

## create a log file to track progress
logging.basicConfig(filename="logfilename_05112022.log", level=logging.INFO)

In [15]:
## Set global thresholds
threshold = -2
vvOwThreshold = -18
connThreshold = 20

In [168]:
# Descending track algorithm
def run_sub_basin(basin_id, studyStart, studyEnd):
    coverages = None
    coveragesDf = None
    results = []
    s1D = None
    studyArea = unions.filterMetadata('system:index', 'equals', basin_id).first()

    s1D = s1.filterBounds(studyArea.geometry())\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.eq('instrumentMode', 'IW')).filterDate(studyStart, studyEnd).filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    # --------------------------------------------------------------
    
    # Pick the orbit number with most coverage over the studyArea
    orbits = s1D.aggregate_array('relativeOrbitNumber_start').distinct()
    
    def calcCoverage(orbitNum):
        first_image = s1D.filterMetadata('relativeOrbitNumber_start', 'equals', orbitNum).first()
        foot = first_image.get('system:footprint')
        foot = ee.Feature(ee.Geometry(foot).convexHull(1))
        intersection = foot.intersection(studyArea, 1)
        coverage = intersection.area().divide(ee.Feature(studyArea).area())
        return (orbitNum, coverage)

    coverages = orbits.map(calcCoverage).getInfo()
    coveragesDf = pd.DataFrame(coverages, columns=['orbitNum', 'jaccard']).sort_values(by = 'jaccard', ascending=False)

    best_coverage = coveragesDf.iloc[0]['jaccard']
    print('best coverage is orbit number '+ str(int(coveragesDf.iloc[0]['orbitNum'])) + ' with ' + str(best_coverage * 100) + '% of the sub-basin')
    
    s1D = s1D.filterMetadata('relativeOrbitNumber_start', 'equals', int(coveragesDf.iloc[0]['orbitNum']))
    image_dates = s1D.aggregate_array('system:time_start')
    
    
    # Map over images in collection and meadian window filter with 15 meter radius
    def medianFilt(image):
        return image.clip(studyArea).focalMedian(15, 'square', 'meters').copyProperties(image)
    
    ## map median filter over the input collection
    
    s1D = s1D.map(medianFilt)

    # Calculate baseline statistics for annual backscatter
    def minMax(pol):
        
        #Define the collection
        coll = s1D.select([pol])        
        
        #Turn the input polarization element into a string for concatenation
        
        pol = ee.String(pol)
        
        # Calculate min, max, mean, and stdDev backscatter by input polarization
        mn = coll.min().rename([pol.cat('_min')])
        mx = coll.max().rename([pol.cat('_max')])
        mean = coll.mean().rename([pol.cat('_mean')])
        std = coll.reduce(ee.Reducer.stdDev()).rename([pol.cat('_std')])
        
        #Calcaulate z score
        
        dif = mx.subtract(mn).rename([pol.cat('_dif')])
        z = dif.divide(std).rename([pol.cat('_z')])
        return mx.addBands([mn, mean, std, dif, z]).set({'pol': pol}).toFloat()
    
    # Apply minMax to each polarization in the collection
    
    vv_stats = minMax('VV')
    # Make a z-score mask for areas that do not have a minimum two standard deviations below the mean annual backscatter
    vvZMask = vv_stats.select(['VV_z']).gt(2)
    
    
    # Function for defining open water on a per-image basis
    def openWaterClass(image):
        
        #set the image class for GEE
        image = ee.Image(image)
        
        #Classify each image for open water based on the fixed threshold for each polarization
        
        vv = image.select(['VV']).lt(vvOwThreshold)
    
        
        #Construct a mask for "open water" areas that are smaller than the threshold for connected pixels
        
        vvAreaMask = vv.connectedPixelCount(1024, False).gt(connThreshold)
        
        #Apply the mask 
        vv = vv.updateMask(vv.neq(0)).updateMask(vvZMask).updateMask(vvAreaMask)

        # Reduce the image by summing the number of pixels classified as inundated across each landcover type
        
        reduced =vv.reduceRegion(ee.Reducer.sum(), \
                  geometry=studyArea.geometry(), \
                  scale=30, \
                    maxPixels = 1e13)

        return reduced
    
    # Make the image collection a list and map the open water classification routine across the list
    descending = s1D.toList(s1D.size()).map(openWaterClass)
    
    #return the mapped image collection as a list and a list of dats of the image acquisitions
    return descending, image_dates


In [167]:
# This function take a basin ID as it's input and maps the above functions across each year relevant to the Sentinel-1 record
# The function saves a CSV of inundated area time series by landcover types to the local machine

def runBasin(basin_id):
    ee.Initialize()
    
    startTime = time.time()
    
    logging.info('basin ID '+ str(basin_id)+ ' start time: ' + str(startTime))
    print('processing '+ str(basin_id))
    dest_path = os.path.join(results_dir, 'basin_id_' + str(basin_id) +'.csv')
    
    
    # iterate over basin IDs and build a dataframe for each
    run_19 = run_sub_basin(basin_id, '2016-01-01', '2020-01-01')
    data_19 = run_19[0].getInfo()
    i_dates_19 = run_19[1].getInfo()
    

    df0 = pd.DataFrame(data_19)
    df0['dates'] = pd.Series(i_dates_19)
 
    test_time_series = df0
    test_time_series['datetimes'] = pd.to_datetime(test_time_series['dates'], unit='ms')
    test_time_series.set_index('datetimes', inplace=True)
    test_time_series.drop(columns=['dates'], inplace=True)
    
    test_time_series.to_csv(dest_path)
    
    executionTime = (time.time() - startTime)
    
    logging.info('basin ID '+ str(basin_id)+ ' processed in seconds: ' + str(executionTime))
    logging.info('')
    print('basin ID '+ str(basin_id)+ ' processed in seconds: ' + str(executionTime))
    time.sleep(2)
    return 'completed basin id: ' + str(basin_id)


In [154]:
# Generate a list of union IDs
unionIds = unions.aggregate_array('system:index').getInfo()

In [ ]:
# Run the routine over union IDs

for i in tqdm(unionIds[112:]):
    try: 
        runBasin(i)
    except:
        time.sleep(60)
        continue
    
print('Done')

  0%|                                                  | 0/5046 [00:00<?, ?it/s]

processing 00000000000000000070
best coverage is orbit number 150 with 100% of the sub-basin
basin ID 00000000000000000070 processed in seconds: 42.879284143447876


  0%|                                       | 1/5046 [00:45<63:54:39, 45.61s/it]

processing 00000000000000000071
best coverage is orbit number 150 with 100% of the sub-basin
basin ID 00000000000000000071 processed in seconds: 21.298761129379272


  0%|                                       | 2/5046 [01:09<46:03:39, 32.87s/it]

...

processing 00000000000000000b48
best coverage is orbit number 150 with 100% of the sub-basin
basin ID 00000000000000000b48 processed in seconds: 42.227794885635376


 55%|██████████████████▏              | 2777/5046 [24:05:11<24:04:14, 38.19s/it]

processing 00000000000000000b49
best coverage is orbit number 150 with 100% of the sub-basin
basin ID 00000000000000000b49 processed in seconds: 48.09995198249817


 55%|██████████████████▏              | 2778/5046 [24:06:02<26:25:48, 41.95s/it]

processing 00000000000000000b4a
best coverage is orbit number 150 with 100% of the sub-basin
